# GRiTS

In [1]:
import warnings

import mbuild as mb
import numpy as np

from grits import backmap
from grits import CG_Compound

### Coarse-graining

A `CG_Compound` is created using an `mbuild.Compound` and a list of tuples of the form ("bead name", "bead SMILES").

[SMILES](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html) strings are a way of specifying chemical information in a string format; in the `cg_beads` example below we are specifying a thiophene ring to be mapped to a "\_B" bead (B for backbone!) with the string "c1sccc1" and three alkyl carbons to be mapped to an "\_S" bead (S for sidechain!) with the string "CCC". Grits uses [openbabel](http://openbabel.org/docs/current/UseTheLibrary/Python_Pybel.html)) for SMILES/SMARTS matching.

The `CG_Compound` class is built on the `mbuild.Compound` class (more about [mbuild](https://mbuild.mosdef.org/en/stable/)) but has some extra feautures which make it convenient for coarse grain structures. Below we'll visualize the coarse grain structure with the atomisitic structure overlaid and on its own.

In [2]:
p3ht = mb.load("../grits/tests/assets/P3HT_16.mol2")

In [3]:
cg_beads = [("_B", "c1sccc1"), ("_S", "CCC")]

p3ht_colors = {"_B": "blue", "_S": "orange"}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    cg_p3ht = CG_Compound(p3ht, cg_beads)

    cg_p3ht.visualize(color_scheme=p3ht_colors, show_atomistic=True).show()
    cg_p3ht.visualize(color_scheme=p3ht_colors).show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Backmapping
In `bead_dict` we list the smiles string for each bead, all the possible "anchors", i.e., atom indices which will form bonds in the final fine-grained structure, and which atom index to attach to the position restraint. Deciding which indices to use as anchors requires knowledge of the structure which will be generated from the smiles string, but smiles-to-structure generation appears to be systematic, so the atom indices will remain consistent. (An example of how to determine which indices correspond to which atom is shown below.)

In `bond_dict` we list all the possible anchor combinations that could account for the bond between two beads. In the example below a bond between the two "\_B" bead could be between anchor points 0-2 or 2-0, both are equivalent. However in the "\_B" to "\_S" bead bond, the only available anchor point on the "\_B" bead is anchor 4, but the "\_S" bead could use anchor 0 or 2. The `backmap` function will choose the best anchors based on distance.

Below is an example showing how to visualize the anchors (particle indices 0,2,and 4) on a thiophene ring. The anchor atoms will show as pink for the thiophene-thiophene anchors and green for the thiophene-sidechain anchor

In [4]:
thiophene = mb.load("c1sccc1", smiles=True)

thiophene[0].name = "X"
thiophene[2].name = "X"
thiophene[4].name = "Cl"

thiophene.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
bead_dict = {
    "_B": {"smiles": "c1sccc1", "anchors": [0, 2, 4]}, # "posres": 1},
    "_S": {"smiles": "CCC", "anchors": [0, 2]}, # "posres": 1},
}

bond_dict = {
    "_B_B": [(0, 2), (2, 0)],
    "_B_S": [(4, 0), (4, 2)],
    "_S_S": [(2, 0), (0, 2)],
}

fine_grained = backmap(cg_p3ht, bead_dict, bond_dict)
fine_grained.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol